In [53]:
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from IPython import embed

def hard_example_mining(dist_mat, labels, return_inds=False):
    #先判断距离矩阵是不是二维，若不是二维则报错
    assert len(dist_mat.size()) == 2
    #判断距离矩阵是否是方阵，若不是方阵则报错
    assert dist_mat.size(0) == dist_mat.size(1)
    N = dist_mat.size(0)#获取方阵长度
    is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())#正样本区掩码，负样本区元素为0
    is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())#负样本区掩码，正样本区元素为0
    
    #计算最小相似度（最大距离）正样本距离与最小相似度所对应正样本的序号（序号范围0~n-1）
    """
    .contiguous()用于将正样本区的距离拉成一维连续向量
    .view(N,-1)用于按照N为行形成矩阵
    torch.max函数不仅返回每一列中最大值的那个元素，并且返回最大值对应索引
    """
    dist_ap, relative_p_indexs = torch.max(dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
    
    #计算最大相似度（最小距离）负样本距离与最大相似度所对应负样本的序号（序号范围0~n-1）
    dist_an, relative_n_indexs = torch.min(
    dist_mat[is_neg].contiguous().view(N, -1), 1, keepdim=True)
    
    #上面计算得到的dist_ap与dist_an维度为(batch_size,1)需要将最后一维进行压缩
    dist_ap = dist_ap.squeeze(1)
    dist_an = dist_an.squeeze(1)
    
    #根据计算得到的序号计算最小相似度正样本与最大相似度负样本在距离矩阵中的序号
    if return_inds:
        indexs = (labels.new().resize_as_(labels).copy_(torch.arange(0, N).long()).unsqueeze( 0).expand(N, N))
        """
        gather函数的用法torch.gather(input, dim, index, out=None) 
        就是从index中找到某值，作为input的某一维度的索引，取出的input的值作为output的某一元素
        核心思想：
        out[i][j][k] = input[index[i][j][k]] [j][k]  # if dim == 0
        out[i][j][k] = input[i][index[i][j][k]][k]   # if dim == 1
        out[i][j][k] = input[i][j][index[i][j][k]]   # if dim == 2
        注，output的维度同index一样
        
        对于下式就是p_indexs[i][0]=indexs[is_neg].contiguous().view(N, -1)[i][relative_n_indexs.data[i][0]]
        因为index即relative_n_indexs.data最后一维值为1，所以只能等于0
        """
        #计算最小相似度正样本与最大相似度负样本在距离矩阵中的序号，结果维度(batch_size,1)
        p_indexs = torch.gather(indexs[is_pos].contiguous().view(N, -1), 1, relative_p_indexs.data)
        n_indexs = torch.gather(indexs[is_neg].contiguous().view(N, -1), 1, relative_n_indexs.data)
        
        #将结果最后一维压缩
        p_indexs = p_indexs.squeeze(1)
        n_indexs = n_indexs.squeeze(1)
        print(dist_ap)
        print(p_indexs)
        return dist_ap, dist_an, p_indexs, n_indexs
    return dist_ap, dist_an

In [54]:
class AlignedTripletLoss(nn.Module):
    def __init__(self, margin=0.3):
        super().__init__()
        self.margin=margin
        self.ranking_loss=nn.MarginRankingLoss(margin=margin)
        self.ranking_loss_local=nn.MarginRankingLoss(margin=margin)
        
    def forward(self, inputs, local_features, targets):
        n=inputs.size(0)
        local_features=local_features.squeeze()
        distance=torch.pow(inputs,2).sum(dim=1, keepdim=True).expand(n,n)
        distance=distance+distance.t()
        #下式等于distance-2inputs*inputs.T
        distance.addmm(1,-2,inputs,inputs.t())
        distance=distance.clamp(min=1e-12).sqrt()
        #获取正负样本对距离，使用最短路径算法
        dist_ap,dist_an,p_inds,n_inds=hard_example_mining(distance,targets,return_inds=True)
if __name__=='__main__':
    target=[1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5,6,6,6,6,7,7,7,7,8,8,8,8]
    target=torch.Tensor(target)
    features=torch.rand(32,2048)
    local_features=torch.randn(32,128,3,1)
    a=AlignedTripletLoss()
    a.forward(features, local_features, target)
    

tensor([37.5999, 37.5814, 37.1423, 37.0747, 37.1946, 37.1577, 37.0481, 37.2760,
        37.1494, 37.3966, 37.0576, 37.0899, 37.5230, 37.9011, 37.5630, 37.1619,
        37.5795, 36.9623, 37.1391, 37.5911, 36.8389, 36.8374, 36.9727, 37.0010,
        37.3449, 36.7780, 37.4706, 37.4769, 37.0633, 37.0867, 37.1064, 37.0244])
tensor([ 0.,  0.,  0.,  0.,  7.,  7.,  7.,  7.,  9.,  9.,  9.,  9., 13., 13.,
        13., 13., 19., 19., 19., 19., 23., 23., 23., 23., 27., 27., 27., 27.,
        30., 30., 30., 30.])
